In [ ]:
import torch
import torch.optim as optim
import os
import pickle
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from model_definitions import Classnet
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
torch.autograd.set_detect_anomaly(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('datas/NYGC_DDPTP_velocity_x.pickle', 'rb') as f:
    x = pickle.load(f)
with open('datas/NYGC_DDPTP_velocity_class_y.pickle', 'rb') as f:
    y = pickle.load(f)

tensor_x = torch.Tensor(x)
tensor_y = torch.tensor(y,dtype=torch.long)
datas = TensorDataset(tensor_x,tensor_y)

train_size = int(0.8 * len(datas))
test_size = len(datas) - train_size
#train_dataset, test_dataset = torch.utils.data.random_split(datas, [train_size, test_size], generator=torch.Generator().manual_seed(42))
#train = DataLoader(train_dataset,batch_size=128,shuffle=True)
#test = DataLoader(test_dataset,batch_size=128,shuffle=True)

In [ ]:
batch_size = 128
hidden_size = 128
rc_class_size = 4
input_dim = 4
learning_rate = 0.001
epochs = 3000

In [ ]:
net = Classnet(input_dim,hidden_size,RC_class_size=rc_class_size)
net.to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=learning_rate,alpha=0.9,eps=1e-7)

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    train_dataset, test_dataset = torch.utils.data.random_split(tdata, [train_size, test_size], generator=torch.Generator().manual_seed(epoch))
    train = DataLoader(train_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)
    test = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)
    net.train()
    running_loss = []
    for i, data in enumerate(train, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output1 = net(inputs)
        loss = lossfn(output1, labels)
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, np.mean(running_loss)))
    running_loss = []
    net.eval()
    correct = 0
    total = 0
    for i, data in enumerate(test, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        output1 = net(inputs)
        _, predictions = torch.max(output1, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct += 1
            total += 1
    print('Accuracy of the network: %d %%' % (100 * correct / total))

In [ ]:
torch.save(net.state_dict(), "./subnets/NYGC_DDPTP_velocity_classnet.pth")